M0 and M1 issues
Change number of components to see the effect
There should only be one eigenvalue, not two
Alpha vectors should only be two values

\begin{align}
H_\alpha\phi(x)\equiv \bigg(-\frac{d^2}{dx^2}+\alpha x^2\bigg)\phi(x)=\lambda\phi(x),\quad \langle\phi|\phi\rangle=1,
\end{align}

\begin{align}
T\phi(x)=-\frac{d^2}{dx^2}\phi(x),\quad V_\alpha(x)\phi(x)=\alpha x^2\phi(x),\quad H_\alpha(x)=T+V(x).
\end{align}


\begin{align}
\bigg(-\frac{d^2}{dx^2}+\alpha x^2-\lambda\bigg)\phi(x)\equiv F_\alpha(\phi(x))=0,
\end{align}


\begin{align}
F_\alpha(\phi(x))=0,\quad\langle\phi|\phi\rangle=1.
\end{align}

\begin{align}
	\hat{\phi}_{\alpha_{k}}(x) = \sum_{i=1}^{n} a_{i} \phi_{i}(x) .
\end{align}

Because we are solving an eigenvalue problem, we can arrive at a set of different-looking equations, using the same set of judges $\{\psi_i(x)\}_{i=1}^n$ as before. We can simply plug $\hat{\phi}_{\alpha_k}$ into the Schrodinger equation and project both sides onto the judges, writing

\begin{align}
\sum_{i=1}^na_i\langle \psi_j|H_{\alpha_k}|\phi_i\rangle=\lambda\sum_{i=1}^na_i\langle\psi_j|\phi_i\rangle.
\end{align}

Define now two matrices:

\begin{align}
M_{ij}(\alpha)\equiv \langle\psi_j|H_\alpha|\phi_i\rangle,\quad N_{ij}\equiv\langle\psi_j|\phi_i\rangle.
\end{align}

These are both $n\times n$ matrices, and we now have a generalized eigenvalue problem for $\vec{a}$:

\begin{align}
M(\alpha)\vec{a}=\lambda N\vec{a}.
\end{align}

Situationally, this may be quicker to solve than finding the roots of the nonlinear system that results from the "traditional" RBM approach.

As with traditional RBM approaches, this is only helpful if we can evaluate $M(\alpha)$ quickly for different $\alpha$ values. For the HO, this is not too hard: we can write

\begin{align}
\langle\psi_j|H_\alpha|\phi_i\rangle=\langle\psi_j|T|\phi_i\rangle+\alpha\langle \psi_j|x^2|\phi_i\rangle\equiv M_{ij}^{(0)}+\alpha M_{ij}^{(1)}.
\end{align}

Our eigenvalue equation is then

\begin{align}
[M^{(0)}+\alpha M^{(1)}]\vec{a}=\lambda N\vec{a},
\end{align}

and all of $M^{(0)},M^{(1)}$ and $N$ can be precomputed.

In [31]:
import numpy as np
import scipy as sci
from scipy import linalg

In [32]:
# Second Derivative
# Potential Matrix
# Components 
# Alphas 
# Projecting Functions (Components) 

def second_derivative_matrix(xgrid):
    N = len(xgrid)
    dx = xgrid[1] - xgrid[0]

    # Generate the matrix for the second derivative using a five-point stencil
    main_diag = np.ones(N) * (-5.0 / 2 / dx**2)
    off_diag = np.ones(N - 1) * 4 / 3 / dx**2
    off_diag2 = np.ones(N - 2) * (-1.0 / (12 * dx**2))

    D2 = np.diag(main_diag) + np.diag(off_diag, k=1) + np.diag(
      off_diag, k=-1) + np.diag(off_diag2, k=2) + np.diag(off_diag2, k=-2)

    return D2

def potential_matrix(xgrid):
    return np.diag(xgrid**2)

def H_creator(alpha, xgrid):
    d2 = second_derivative_matrix(xgrid)
    pot = potential_matrix(xgrid)
    H = d2 + alpha*pot
    return H

def hf_solve(H):
    evals, evects = np.linalg.eigh(H)
    return evals, evects


In [33]:
alphas = [.5,2,5]
x_max = 10.0
h = 10**(-1)
x = np.arange(-x_max, x_max + h, h)
m = np.zeros((len(alphas), x.shape[0]))

for i in range(len(alphas)):
    alpha = alphas[i]
    H = H_creator(alpha, x)
    evals, evects = hf_solve(H)
    # m[i] = evects[0] / np.linalg.norm(evects[0])*np.sign(evects[0][  int(len(x)/2)  ])
    m[i] = evects[:,i] 

In [34]:
U, sigma, Vh = np.linalg.svd(m)
#components = 3
components = len(m)
print(components)
reduced_basis = Vh[:components]

reduced_basis = [reduced_basis[i]*np.sign(reduced_basis[i][  int(len(x)/2)  ]) for i in range(len(reduced_basis))]

3


In [35]:
# psi = np.array(reduced_basis)
# phi = np.array(reduced_basis)
psi = m
phi = m

In [36]:
d2 = second_derivative_matrix(x)
pot = potential_matrix(x)

In [37]:
# def M0(psi, phi, d2, i, j):
#     inner_product = np.dot(psi[j], phi[i])
#     result_vector = np.dot(d2, psi[j])
#     M0 = np.dot(result_vector, phi[i])
#     return M0

def M0(psi, phi, d2, i, j):
    M0 = np.dot(psi[j], np.dot(d2, phi[i]))
    return M0

In [38]:
# def M1(psi, phi, pot, i, j):
#     inner_product = np.dot(psi[j], phi[i])
#     result_vector = np.dot(pot, psi[j])
#     M1 = np.dot(result_vector, phi[i])
#     return M1

def M1(psi, phi, pot, i, j):
    M1 = np.dot(psi[j], np.dot(pot, phi[i]))
    return M1

In [39]:
compvec = np.zeros(components)
array = []
for i in range(components):
    array.append(compvec)
H_hat = np.array(array)
N = np.array(array)


[ ⟨𝜓₀|d2|𝜙₀⟩ + α⟨𝜓₁|pot|𝜙₀⟩   ⟨𝜓₁|d2|𝜙₀⟩ + α⟨𝜓₁|pot|𝜙₀⟩   ⟨𝜓₂|d2|𝜙₀⟩ + α⟨𝜓₂|pot|𝜙₀⟩ ]
[ ⟨𝜓₀|d2|𝜙₁⟩ + α⟨𝜓₀|pot|𝜙₁⟩   ⟨𝜓₁|d2|𝜙₁⟩ + α⟨𝜓₁|pot|𝜙₁⟩   ⟨𝜓₂|d2|𝜙₁⟩ + α⟨𝜓₂|pot|𝜙₁⟩ ]
[ ⟨𝜓₀|d2|𝜙₂⟩ + α⟨𝜓₀|pot|𝜙₂⟩   ⟨𝜓₁|d2|𝜙₂⟩ + α⟨𝜓₁|pot|𝜙₂⟩   ⟨𝜓₂|d2|𝜙₂⟩ + α⟨𝜓₂|pot|𝜙₂⟩ ]

In [50]:
def create_H_hat(alpha, phi, psi, pot, d2):
    for i in range(components):
        for j in range(len(H_hat[i])):
            H_hat[i][j] = M0(psi, phi, d2, i, j) + alpha*M1(psi,phi,pot,i,j)
    return H_hat

In [51]:
H_hat = create_H_hat(5, phi, psi, pot, d2)
print(H_hat)

[[-5.28317544e+02  1.04328542e-11 -1.76157357e+02]
 [ 1.04319227e-11 -5.23773215e+02 -1.36391246e-11]
 [-1.76157357e+02 -1.36499562e-11 -5.18964718e+02]]


In [52]:
def create_N(psi, phi):
    for i in range(len(N[0])):
        for j in range(len(N[1])):
            N[i,j] = np.dot(psi[j], phi[i])
    return N

In [53]:
N = create_N(psi, phi)
print(N)
for i in range(len(N)):
    for j in range(len(N[i])):
        if N[i][j] < .99:
            N[i][j] = 0
print(N)

[[ 1.00000000e+00 -1.98027358e-14  3.39439949e-01]
 [-1.98027358e-14  1.00000000e+00  2.62637134e-14]
 [ 3.39439949e-01  2.62637134e-14  1.00000000e+00]]
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


In [54]:
def solve(N, H_hat):
    print(N)
    print(H_hat)
    evals, evects = sci.linalg.eigh(H_hat, b = N)
    eigenvalue = evals[0]
    return eigenvalue, evects

eval, a_vecs = solve(N, H_hat)
print(eval)

[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
[[-5.28317544e+02  1.04328542e-11 -1.76157357e+02]
 [ 1.04319227e-11 -5.23773215e+02 -1.36391246e-11]
 [-1.76157357e+02 -1.36499562e-11 -5.18964718e+02]]
-699.8605495298534


In [55]:
def ExactEigenvalue(alpha):
    return 2 * (.5) * np.sqrt(alpha / 1**2)
print(ExactEigenvalue(5))

2.23606797749979
